In [9]:
pip install git+https://github.com/huggingface/transformers accelerate

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /localscratch-nvme/3179674/pip-req-build-xio1c0oq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /localscratch-nvme/3179674/pip-req-build-xio1c0oq
  Resolved https://github.com/huggingface/transformers to commit e6a8063ef1af16df964b644b07e1d17e96555d23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [10]:
!pip install torch
!pip install qwen_vl_utils

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [11]:
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable


In [12]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import tensorflow as tf
print("TensorFlow GPU available:", len(tf.config.list_physical_devices('GPU')) > 0)
print("Devices:", tf.config.list_physical_devices('GPU'))

2025-07-04 10:41:56.499752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751640116.943940 3925795 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751640117.054188 3925795 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751640118.138210 3925795 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751640118.138232 3925795 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751640118.138234 3925795 computation_placer.cc:177] computation placer alr

TensorFlow GPU available: True
Devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [14]:
import torch
from transformers import (
    LlavaForConditionalGeneration, AutoProcessor as LlavaProcessor,
    Qwen2_5_VLForConditionalGeneration, AutoProcessor as QwenProcessor
)
from qwen_vl_utils import process_vision_info
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor

from datasets import load_dataset
from PIL import Image
import requests
from io import BytesIO

In [15]:
llava_model = LlavaForConditionalGeneration.from_pretrained(
    "llava-hf/llava-1.5-7b-hf", torch_dtype=torch.float16, device_map="auto"
)
llava_processor = LlavaProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

qwen_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)
qwen_processor = QwenProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [8]:
data_sqa = load_dataset('derek-thomas/ScienceQA', split='test')

In [9]:
# dataset = load_dataset("lmms-lab/ai2d", split="test[:1000]")  

In [10]:
print(data_sqa[5])

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=760x506 at 0x1553C91EB690>, 'question': 'Which of these organisms contains matter that was once part of the lichen?', 'choices': ['bilberry', 'mushroom'], 'answer': 1, 'hint': 'Below is a food web from a tundra ecosystem in Nunavut, a territory in Northern Canada.\nA food web models how the matter eaten by organisms moves through an ecosystem. The arrows in a food web represent how matter moves between organisms in an ecosystem.', 'task': 'closed choice', 'grade': 'grade5', 'subject': 'natural science', 'topic': 'biology', 'category': 'Ecosystems', 'skill': 'Interpret food webs II', 'lecture': 'A food web is a model.\nA food web shows where organisms in an ecosystem get their food. Models can make things in nature easier to understand because models can represent complex things in a simpler way. If a food web showed every organism in an ecosystem, the food web would be hard to understand. So, each food web shows how some or

In [16]:
def chat_with_qwen(data_point):
    image = data_point['image']
    if image is not None:
        
        text = str("question:" + data_point['question'] + "choices:" + str(data_point['choices']))
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": data_point['image'],
                    },
                    {"type": "text", "text": text + "What is your answer? Justify"},
                ],
            }
        ]
        inputs = qwen_processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = qwen_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
        )
    
        device = torch.device("cuda")
        inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
        for k in ["pixel_values", "image_grid_thw", "video_pixel_values", "video_image_size"]:
            if k in inputs:
                del inputs[k]
    
        generated_ids = qwen_model.generate(**inputs, max_new_tokens=5000)
        generated_ids_trimmed = [ out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs["input_ids"], generated_ids) ]
    
        output_text = qwen_processor.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )
    
        return output_text

    
    


In [13]:
# a = chat_with_qwen_for_scienceQA(data_sqa[41])

In [15]:
# print(a)

In [17]:
data_vqa = load_dataset("HuggingFaceM4/VQAv2", split="validation[:1000]")

Repo card metadata block was not found. Setting CardData to empty.


In [35]:
print(data_vqa[3])

{'question_type': 'what', 'multiple_choice_answer': 'foodiebakercom', 'answers': [{'answer': 'foodiebakercom', 'answer_confidence': 'yes', 'answer_id': 1}, {'answer': 'foodiebakercom', 'answer_confidence': 'yes', 'answer_id': 2}, {'answer': 'foodiebaker', 'answer_confidence': 'yes', 'answer_id': 3}, {'answer': 'foodiebakercom', 'answer_confidence': 'yes', 'answer_id': 4}, {'answer': 'foodiebakercom', 'answer_confidence': 'yes', 'answer_id': 5}, {'answer': 'http://foodiebakercom', 'answer_confidence': 'yes', 'answer_id': 6}, {'answer': 'foodiebakercom', 'answer_confidence': 'yes', 'answer_id': 7}, {'answer': 'foodiebakercom', 'answer_confidence': 'yes', 'answer_id': 8}, {'answer': 'foodiebakercom', 'answer_confidence': 'yes', 'answer_id': 9}, {'answer': 'foodiebaker', 'answer_confidence': 'yes', 'answer_id': 10}], 'image_id': 393225, 'answer_type': 'other', 'question_id': 393225000, 'question': 'What website copyrighted the picture?', 'image': <PIL.JpegImagePlugin.JpegImageFile image mo

In [19]:
from PIL import Image

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
def chat_with_qwen_vqav2(data_point):
    image = data_point['image']
    if image is None:
        return "No image provided."

    question = data_point['question']
    
    # Extract unique answers as choices (e.g., ["watching", "spectating"])
    all_answers = [a["answer"].strip().lower() for a in data_point["answers"]]

    
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {
                    "type": "text",
                    "text": question + " Instruction: strictly answer only in a word",
                },
            ],
        }
    ]

    text = qwen_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = qwen_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

    generated_ids = qwen_model.generate(**inputs, max_new_tokens=1000)
    trimmed = [out[len(inp):] for inp, out in zip(inputs["input_ids"], generated_ids)]
    output_text = qwen_processor.batch_decode(trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]
    pred = output_text.strip().lower()
    return pred


In [22]:
output = chat_with_qwen_vqav2(data_vqa[1])
print(output)

watching


In [23]:
def chat_with_llava_vqav2(data_point):
    image = data_point['image']
    if image is None:
        return "No image provided."

    question = data_point['question']
    
    all_answers = [a["answer"].strip().lower() for a in data_point["answers"]]

    
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {
                    "type": "text",
                    "text": question + " Instruction: strictly answer only in a word",
                },
            ],
        }
    ]
    inputs = llava_processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
    ).to(llava_model.device, torch.float16)

    inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

    generate_ids = llava_model.generate(**inputs, max_new_tokens=1000)
    trimmed = [out[len(inp):] for inp, out in zip(inputs["input_ids"], generate_ids)]
    output_text = llava_processor.batch_decode(trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]
    pred = output_text.strip().lower()

    return pred

In [24]:
output = chat_with_llava_vqav2(data_vqa[1])
print(output)

watching


In [58]:
def build_initial_context(agent_name, data_point):
    """Initial prompt with image and question"""
    return [{
        "role": "user",
        "content": [
            {"type": "image", "image": data_point["image"]},
            {"type": "text", "text": f"Question: {data_point['question']}\nPlease explain your answer."}
        ]
    }]

def build_peer_message(question, peer_responses, image, own_last_response=None):
    """Builds a user message listing other agents' prior responses and discouraging repetition."""
    context = "Other agent responses:\n"
    for response in peer_responses:
        context += f"\nOne agent said:\n{response.strip()}\n"

    context += f"\nThe original question is: {question}\n"
    if own_last_response:
        context += f"Previously, you answered: '{own_last_response}'. Do you still think that you are correct? Justify your answer and point out why the other agent might be wrong. Do not repeat yourself or simply copy the other agent's answer but give a reasoning.\n"

    context += "Please explain your reasoning again."
    return {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": context}
        ]
    }


def generate_response(model, processor, chat_history, image):
    """Runs chat model with full history"""
    text = processor.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(chat_history)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = {k: v.to(model.device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
    outputs = model.generate(**inputs, max_new_tokens=512)
    trimmed = [out[len(inp):] for inp, out in zip(inputs["input_ids"], outputs)]
    response = processor.batch_decode(trimmed, skip_special_tokens=True)[0].strip()
    return response

def ask_final_answer(model, processor, data_point, prior_response):
    """Asks for final one-word answer using model and context"""
    prompt = f"Question: {data_point['question']}\nEarlier you said:\n{prior_response}\n\nWhat is your final answer? Respond with only one word."
    messages = [{
        "role": "user",
        "content": [
            {"type": "image", "image": data_point["image"]},
            {"type": "text", "text": prompt}
        ]
    }]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = {k: v.to(model.device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
    outputs = model.generate(**inputs, max_new_tokens=10)
    trimmed = [out[len(inp):] for inp, out in zip(inputs["input_ids"], outputs)]
    return processor.batch_decode(trimmed, skip_special_tokens=True)[0].strip().lower()


In [59]:
def vqav2_debate(data_point, rounds=3, verbose=True):
    qwen_history = build_initial_context("Qwen", data_point)
    llava_history = build_initial_context("LLaVA", data_point)
    debate_log = []

    qwen_prev_response = None
    llava_prev_response = None

    for r in range(1, rounds + 1):

        # Qwen sees LLaVA's prior + discouragement to repeat
        if r > 1:
            qwen_msg = build_peer_message(
                question=data_point["question"],
                peer_responses=[llava_prev_response],
                image=data_point["image"],
                own_last_response=qwen_prev_response
            )
            qwen_history.append(qwen_msg)

        qwen_response = generate_response(qwen_model, qwen_processor, qwen_history, data_point["image"])
        qwen_history.append({"role": "assistant", "content": qwen_response})
        debate_log.append({"role": f"Qwen (Round {r})", "content": qwen_response})

        qwen_prev_response = qwen_response

        # LLaVA sees Qwen’s prior + discouragement to repeat
        if r > 1:
            llava_msg = build_peer_message(
                question=data_point["question"],
                peer_responses=[qwen_prev_response],
                image=data_point["image"],
                own_last_response=llava_prev_response
            )
            llava_history.append(llava_msg)

        llava_response = generate_response(llava_model, llava_processor, llava_history, data_point["image"])
        llava_history.append({"role": "assistant", "content": llava_response})
        debate_log.append({"role": f"LLaVA (Round {r})", "content": llava_response})
        llava_prev_response = llava_response

        # Ask for final answers (after reasoning)
        qwen_final = ask_final_answer(qwen_model, qwen_processor, data_point, qwen_response)
        llava_final = ask_final_answer(llava_model, llava_processor, data_point, llava_response)
        debate_log.append({"role": "Qwen (Final)", "content": qwen_final})
        debate_log.append({"role": "LLaVA (Final)", "content": llava_final})

        if qwen_final == llava_final:
            debate_log.append({"role": "Consensus", "content": f"Both agreed on: '{qwen_final}' after {r} round(s)."})
            return {
                "question_id": data_point["question_id"],
                "multiple_choice_answer": data_point["multiple_choice_answer"],
                "question": data_point["question"],
                "image_id": data_point.get("image_id", None),
                "answer": qwen_final,
                "rounds": r,
                "consensus": True,
                "history": debate_log
            }
        
    debate_log.append({"role": "Disagreement", "content": f"No consensus reached after {rounds} rounds."})
    return {
        "question_id": data_point["question_id"],
        "multiple_choice_answer": data_point["multiple_choice_answer"],
        "question": data_point["question"],
        "image_id": data_point.get("image_id", None),
        "answer": None,
        "rounds": None,
        "consensus": False,
        "history": debate_log
    }





In [60]:
res = []

In [61]:
print(len(data_vqa))

1000


In [62]:

for i in data_vqa:
    debate_result = vqav2_debate(i)
    res.append(debate_result)
    if len(res)==250 or len(res)==750:
        print(len(res))
    


250
750


In [63]:
import json
with open("vqav2_debate_results2.json", "w") as f:
    json.dump(res, f, indent=2)

In [64]:
ls -lh vqav2_debate_results.json


-rw-r--r-- 1 apoorvachavali apoorvachavali 1.3M Jul  4 10:40 vqav2_debate_results.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [65]:
baseline_agree = 0
baseline_total = 0
baseline_agree_correct = 0
baseline_agree_wrong = 0
baseline_qwen_correct = 0
baseline_llava_correct = 0

In [66]:
for i in data_vqa:
    a = chat_with_qwen_vqav2(i).strip().lower()
    b = chat_with_llava_vqav2(i).strip().lower()
    gt = i["multiple_choice_answer"].strip().lower()
    
    if a == b:
        baseline_agree+=1
        if a == gt:
            baseline_agree_correct+=1
        else:
            baseline_agree_wrong+=1

    if a == gt:
        baseline_qwen_correct+=1
    if b == gt:
        baseline_llava_correct+=1
    baseline_total+=1

In [67]:
print(f"Total: {baseline_total}")
print(f"Qwen correct: {baseline_qwen_correct} ({baseline_qwen_correct / baseline_total:.2%})")
print(f"LLaVA correct: {baseline_llava_correct} ({baseline_llava_correct / baseline_total:.2%})")
print(f"Models agreed: {baseline_agree} ({baseline_agree / baseline_total:.2%})")
print(f"Agreement correct: {baseline_agree_correct} ({baseline_agree_correct / baseline_agree:.2%})" if baseline_agree else "No agreement")
print(f"Agreement wrong: {baseline_agree_wrong}")


Total: 1000
Qwen correct: 736 (73.60%)
LLaVA correct: 663 (66.30%)
Models agreed: 681 (68.10%)
Agreement correct: 590 (86.64%)
Agreement wrong: 91


In [69]:
import json

with open('vqav2_debate_results2.json', 'r') as f:
    json_data = json.load(f)

In [79]:
qwen_llava_agree = 0
qwen_llava_agree_correct = 0
qwen_llava_agree_wrong = 0

In [80]:
print(json_data[1])

{'question_id': 262148001, 'multiple_choice_answer': 'watching', 'question': 'What are the people in the background doing?', 'image_id': 262148, 'answer': 'watching', 'rounds': 1, 'consensus': True, 'history': [{'role': 'Qwen (Round 1)', 'content': 'The people in the background appear to be watching the skateboarder perform a trick. They are standing at a safe distance, observing the action. Some of them might be waiting for their turn or simply enjoying the performance. The presence of onlookers suggests that this is likely a skateboarding event or a casual gathering where people come to watch and support the skateboarders.'}, {'role': 'LLaVA (Round 1)', 'content': "The people in the background are watching the skateboarder perform his tricks. They are likely observing the skateboarder's skills and enjoying the show."}, {'role': 'Qwen (Final)', 'content': 'watching'}, {'role': 'LLaVA (Final)', 'content': 'watching'}, {'role': 'Consensus', 'content': "Both agreed on: 'watching' after 1

In [81]:
for i in json_data:
    if i['consensus']== True:
        qwen_llava_agree+=1
        if i['answer'] == i['multiple_choice_answer']:
            qwen_llava_agree_correct+=1
        else:
            qwen_llava_agree_wrong+=1

In [83]:
print("Qwen and llava agree on ", qwen_llava_agree)
print("Qwen and llava agree on and match with GT", qwen_llava_agree_correct)
print("Qwen and llava agree on and does not match with GT", qwen_llava_agree_wrong)

Qwen and llava agree on  824
Qwen and llava agree on and match with GT 627
Qwen and llava agree on and does not match with GT 197
